# NETA Mining Distribution - Effects on NETA Rich List
The NETA subpool on GetBlok has been running for about 2 weeks now.  

Since this is about the halfway point (a little over half the NETA has been depleted) I was curious what effect, if any the subpool has had on NETA distribution & the NETA "Rich List."

### Download the Data

The [Ergo Tokenautics](https://github.com/babygrenade/ergo-tokenautics) tool captures daily snapshots of richlists for various tokens.  Using the [frontend](https://babygrenade.github.io/ergo-tokenautics-frontend/?token=NETA) we can select "snapshots" for today and 6/11 (the day the NETA subpool launched) and get urls for the raw csv files.

I download the data below and convert them to pandas dataframes.

In [166]:
import requests
import pandas as pd
from io import StringIO

current = 'https://raw.githubusercontent.com/babygrenade/ergo-tokenautics/a0b8e3726fc2e24a0a67ddb2a9d530794ebc51cd/data/NETA.csv'
old = 'https://raw.githubusercontent.com/babygrenade/ergo-tokenautics/e2ba650c52531c9268002ddce10e12a92d98e8e4/data/NETA.csv'
r = requests.get(current)
current_df = pd.read_csv(StringIO(r.text))
r = requests.get(old)
old_df = pd.read_csv(StringIO(r.text))


In [167]:
# Adjusting amounts for NETA's 6 decimal places:
current_df['amount'] = current_df['amount'] / (10**6)
old_df['amount'] = old_df['amount'] / (10**6)

### Add a rank column.

In [168]:
current_df['rank'] = current_df['amount'].rank(ascending=False).astype('int')
old_df['rank'] = old_df['amount'].rank(ascending=False).astype('int')

### Find change in rank.
By joining the history dataset to the current dataset, we find the difference in rank for any addresses.

In [169]:
joined_df = current_df.merge(old_df,how='left', on = 'address', suffixes = ('','_old'))
joined_df['rank_change'] = joined_df['rank_old'].fillna(max(current_df['rank'])+1) - joined_df['rank']
joined_df['rank_change'] = joined_df['rank_change'].astype('int')
joined_df.drop('rank_old',axis=1,inplace=True)
joined_df['amount_change'] = joined_df['amount'] - joined_df['amount_old'].fillna(0)

### Formatting Helper Functions

In [170]:
# Colors negative numbers red and positive numbers green.
def color_negative_red(value):
    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'
    return 'color: %s' % color

In [171]:
# Shortens contract addresses for readability and adds a clickable link to the explorer.
def format_address(val):
    url = 'https://explorer.ergoplatform.com/en/addresses/'+ val
    if len(val)> 51:
        val = val[:50] + '...'
    return f'<a target="_blank" href="{url}">{val[:53]}</a>'

### Display Results
On row 4 we see what looks like the address for the mining subpool contract with about 5 million NETA left in it.

Poking around at some of the gainers - it doesn't look like there are many miners breaking into the top holders list.  
There is a big increase in NETA liquidity on ErgoDEX so my guess is that's where most of the NETA from the Subpool is going, either being sold or as a liquidity pair.

In [172]:
joined_df.rename(columns={'percentage':'%', 'percentage_old':'%_old'},inplace=True)
joined_df.head(100) \
    .style.applymap(color_negative_red,subset=['rank_change','amount_change'])\
    .format({'address': format_address}) \
    .set_properties(**{'font-size': '10pt'}) \
    .set_properties(subset=['address'],**{'font-size': '8pt'})